In [1]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zswd7m96
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zswd7m96
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=8b26f53a35b9d89dca594e6abde64c80b2367b16f3b51ce598d5434cae0fd6d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-jyvskf6g/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
cr

In [9]:
%%sh
cat > pi-mpi.c << EOF

#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// Funkcja do obliczania potęgi
double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int rank; // Numer procesu
    int size; // Liczba procesów
    int tag = 1;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    clock_t t;
    double time_taken;

    if(rank == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0;

      for(int i=1; i<size; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        lpi +=x;
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        total_time+=t;
      }
      printf("Wartość liczby pi = %f \n", 4 * lpi);
      printf("Całkowity czas wykonywania = %f \n", total_time);
    }

    if(rank != 0)
    {
      t = clock();
      // Obliczanie wyniku dla danego procesu
      double x = potegowanie( -1, rank - 1) / ( 2 * rank - 1); 
      printf("Wynik procesu %d = %f \t" , rank , x);
      // Wysyłanie wyniku do procesu 0
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      t = clock() - t;
      time_taken = ((double)t)/CLOCKS_PER_SEC;
      // Wysyłanie informacji o czasie wykonania do procesu 0
      MPI_Send(&time_taken, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu %d = %f\n",rank,time_taken);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 60 --allow-run-as-root a.out

Wynik procesu 34 = -0.014925 	Czas wykonywania sie procesu 34 = 0.000096
Wynik procesu 39 = 0.012987 	Czas wykonywania sie procesu 39 = 0.000051
Wynik procesu 42 = -0.012048 	Czas wykonywania sie procesu 42 = 0.000039
Wynik procesu 57 = 0.008850 	Czas wykonywania sie procesu 57 = 0.000043
Wynik procesu 25 = 0.020408 	Czas wykonywania sie procesu 25 = 0.000037
Wynik procesu 13 = 0.040000 	Czas wykonywania sie procesu 13 = 0.000035
Wynik procesu 7 = 0.076923 	Czas wykonywania sie procesu 7 = 0.000037
Wynik procesu 6 = -0.090909 	Czas wykonywania sie procesu 6 = 0.000031
Wynik procesu 12 = -0.043478 	Czas wykonywania sie procesu 12 = 0.000037
Wynik procesu 31 = 0.016393 	Czas wykonywania sie procesu 31 = 0.000034
Wynik procesu 9 = 0.058824 	Czas wykonywania sie procesu 9 = 0.000033
Wynik procesu 30 = -0.016949 	Czas wykonywania sie procesu 30 = 0.000034
Wynik procesu 46 = -0.010989 	Czas wykonywania sie procesu 46 = 0.000024
Wynik procesu 21 = 0.024390 	Czas wykonywania sie procesu 21 = 0

In [17]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// Funkcja, którą calkujemy
double funkcja(double x)
{
  return (x * x + 2 * x + 3*x);
}

int main(int argc, char **argv)
{
  // Inicjalizacja zmiennych potrzebnych do obsługi MPI
    int rank; // Numer procesu
    int size; // Liczba procesów
    int tag=1; // Etykieta wiadomości
    
    MPI_Status status; // Zmienna przechowująca status odebrania wiadomości
    MPI_Init(&argc, &argv); // Inicjalizacja MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Pobranie numeru procesu
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Pobranie liczby procesów
    clock_t t; // Zmienna do mierzenia czasu
    double time_taken; // Zmienna do przechowywania czasu wykonania procesu
    // Zmienne do przechowywania liczby procesów i numeru procesu w formacie double
    double numproc = size, nmbproc = rank;
    // Granice calkowania
    double lower = 1,upper = 4;
    // Krok calkowania
    double h = (upper - lower) / numproc;
    // Proces o numerze 0 jest głównym procesem i odbiera wyniki od pozostałych procesów
    if(rank == 0)
    {
      double calka = 0 ,x=0;
      // Odbieranie wyników od pozostałych procesów
      for(int i=1; i<size; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=x;
      }
      // Wypisanie wyniku
      printf("Wartosc dolnej granicy calki = %f \n",funkcja(lower)/2 * h);
      printf("Wartosc gornej granicy calki = %f \n",funkcja(upper)/2 * h);

      printf("Wartosc calki = %1f \n", h * (funkcja(lower)/2 + calka 
        + funkcja(upper)/2));
    }

    if(rank != 0)
    {
        t = clock();
        double xi,wynik;
        nmbproc = rank;
        xi = lower + (nmbproc/numproc) 
            * (upper - lower);
        wynik = funkcja(xi);
        printf("Wynik  %d = %f \t", rank,wynik*h);
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
        t = clock() - t;
        time_taken = ((double)t)/CLOCKS_PER_SEC;
        printf("Czas wykonywania:  %d = %f\n",rank,time_taken);
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 100 --allow-run-as-root a.out

Wynik  56 = 0.617472 	Czas wykonywania:  56 = 0.000062
Wynik  83 = 0.888903 	Czas wykonywania:  83 = 0.000113
Wynik  89 = 0.954567 	Czas wykonywania:  89 = 0.000059
Wynik  30 = 0.393300 	Czas wykonywania:  30 = 0.000040
Wynik  95 = 1.022175 	Czas wykonywania:  95 = 0.000051
Wynik  33 = 0.417303 	Czas wykonywania:  33 = 0.000050
Wynik  22 = 0.331668 	Czas wykonywania:  22 = 0.000048
Wynik  40 = 0.475200 	Czas wykonywania:  40 = 0.000049
Wynik  18 = 0.302148 	Czas wykonywania:  18 = 0.000058
Wynik  23 = 0.339183 	Czas wykonywania:  23 = 0.000045
Wynik  7 = 0.225423 	Czas wykonywania:  7 = 0.000044
Wynik  82 = 0.878148 	Czas wykonywania:  82 = 0.000041
Wynik  99 = 1.068327 	Czas wykonywania:  99 = 0.000045
Wynik  81 = 0.867447 	Czas wykonywania:  81 = 0.000046
Wynik  77 = 0.825183 	Czas wykonywania:  77 = 0.000067
Wynik  76 = 0.814752 	Czas wykonywania:  76 = 0.000052
Wynik  94 = 1.010772 	Czas wykonywania:  94 = 0.000052
Wynik  17 = 0.294903 	Czas wykonywania:  17 = 0.000258
Wynik  21 = 